In [19]:
!git clone https://github.com/fayrose/EgyptianTranslation.git

fatal: destination path 'EgyptianTranslation' already exists and is not an empty directory.


In [20]:
!pip install OpenNMT-py

In [21]:
string = """
# egy_eng.yaml

## Where the samples will be written
save_data: EgyptianTranslation/onmt-files/run/example
## Where the vocab(s) will be written
src_vocab: EgyptianTranslation/onmt-files/run/example.vocab.egy
tgt_vocab: EgyptianTranslation/onmt-files/run/example.vocab.eng
# Prevent overwriting existing files in the folder
overwrite: True

# Corpus opts:
data:
    corpus_1:
        path_src: EgyptianTranslation/compiled_corpora/aligned_train.egy.csv
        path_tgt: EgyptianTranslation/compiled_corpora/aligned_train.eng.csv
    valid:
        path_src: EgyptianTranslation/compiled_corpora/aligned_val.egy.csv
        path_tgt: EgyptianTranslation/compiled_corpora/aligned_val.eng.csv
"""
with open("egy_eng.yaml", 'w') as yaml_file:
  yaml_file.write(string)

In [22]:
!onmt_build_vocab -config egy_eng.yaml -n_sample 15000

Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-07-02 03:26:48,927 INFO] Counter vocab from 15000 samples.
[2021-07-02 03:26:48,927 INFO] Build vocab on 15000 transformed examples/corpus.
[2021-07-02 03:26:48,936 INFO] corpus_1's transforms: TransformPipe()
[2021-07-02 03:26:48,937 INFO] Loading ParallelCorpus(EgyptianTranslation/compiled_corpora/aligned_train.egy.csv, EgyptianTranslation/compiled_corpora/aligned_train.eng.csv, align=None)...
[2021-07-02 03:26:49,038 INFO] Counters src:2662
[2021-07-02 03:26:49,038 INFO] Counters tgt:4402
[2021-07-02 03:26:49,038 WARNING] path EgyptianTranslation/onmt-files/run/example.vocab.egy exists, may overwrite...
[2021-07-02 03:26:49,040 WARNING] path EgyptianTranslation/onmt-files/run/example.vocab.eng exists, may overwrite...


In [23]:
string = """
# Vocabulary files that were just created
src_vocab: EgyptianTranslation/onmt-files/run/example.vocab.egy
tgt_vocab: EgyptianTranslation/onmt-files/run/example.vocab.eng

# Train on a single GPU
world_size: 1

# Where to save the checkpoints
save_model: EgyptianTranslation/onmt-files/run/model
save_checkpoint_steps: 1000
train_steps: 3000
valid_steps: 1500

# Optimization
model_dtype: "fp32"
optim: "adam"
learning_rate: 2
warmup_steps: 8000
decay_method: "noam"
adam_beta2: 0.998
max_grad_norm: 0
param_init: 0
param_init_glorot: true
label_smoothing: 0.1
normalization: "tokens"

# Model
encoder_type: transformer
decoder_type: transformer
position_encoding: true
heads: 8
rnn_size: 512
word_vec_size: 512
max_generator_batches: 2
transformer_ff: 2048
dropout_steps: [0]
dropout: [0.1]
accum_count: 2
attention_dropout: [0.1]
share_vocab: true
gpu_ranks: 0
"""

with open("egy_eng.yaml", 'a') as yaml_file:
  yaml_file.write(string)

In [24]:
!onmt_train --config egy_eng.yaml

[2021-07-02 03:26:49,818 INFO] Missing transforms field for corpus_1 data, set to default: [].
[2021-07-02 03:26:49,818 WARNING] Corpus corpus_1's weight should be given. We default it to 1 for you.
[2021-07-02 03:26:49,819 INFO] Missing transforms field for valid data, set to default: [].
[2021-07-02 03:26:49,819 INFO] Parsed 2 corpora from -data.
[2021-07-02 03:26:49,819 INFO] Get special vocabs from Transforms: {'src': set(), 'tgt': set()}.
[2021-07-02 03:26:49,819 INFO] Loading vocab from text file...
[2021-07-02 03:26:49,819 INFO] Loading src vocabulary from EgyptianTranslation/onmt-files/run/example.vocab.egy
[2021-07-02 03:26:49,824 INFO] Loaded src vocab has 2662 tokens.
[2021-07-02 03:26:49,826 INFO] Loading tgt vocabulary from EgyptianTranslation/onmt-files/run/example.vocab.eng
[2021-07-02 03:26:49,834 INFO] Loaded tgt vocab has 4402 tokens.
[2021-07-02 03:26:49,836 INFO] Building fields with vocab in counters...
[2021-07-02 03:26:49,841 INFO]  * tgt vocab size: 4406.
[2021-

In [25]:
!onmt_translate -model EgyptianTranslation/onmt-files/run/model_step_3000.pt -src /content/EgyptianTranslation/compiled_corpora/aligned_test.egy.csv -output EgyptianTranslation/onmt-files/run/pred_3000.txt -verbose


Streaming output truncated to the last 5000 lines.
SENT 296: ['sAp', '.n', '.j', 'mr']
PRED 296: i stretched out
PRED SCORE: -3.4762

[2021-07-02 03:30:37,954 INFO] 
SENT 297: ['r', 'kt', 'Hr', 'tA']
PRED 297: for another term on earth
PRED SCORE: -0.2129

[2021-07-02 03:30:37,955 INFO] 
SENT 298: ['r', 'jT', 'Tw', 'Smsw', '.j']
PRED 298: took possession of my followers
PRED SCORE: -4.6490

[2021-07-02 03:30:37,955 INFO] 
SENT 299: ['sA', 'ra', 's', 'n', 'wsrt']
PRED 299: son of re sesostris
PRED SCORE: -0.1595

[2021-07-02 03:30:37,955 INFO] 
SENT 300: ['smn', '.n', '.f', 'wDw', 'Sma']
PRED 300: he erected the upper egyptian boundary marker
PRED SCORE: -1.7081

[2021-07-02 03:30:38,803 INFO] 
SENT 301: ['smr', 'wa']
PRED 301: sole companion
PRED SCORE: -0.2124

[2021-07-02 03:30:38,803 INFO] 
SENT 302: ['wat', 'HD', '.tw', '.s']
PRED 302: a single thing has been destroyed
PRED SCORE: -0.7005

[2021-07-02 03:30:38,803 INFO] 
SENT 303: ['wDA', '.f', 'r', 'nD', 'jt', '.f']
PRED 303: when

In [26]:
ref_path = "EgyptianTranslation/compiled_corpora/aligned_test.eng.csv"
with open(ref_path, 'r') as ref_file:
  ref = [list(map(lambda x: x.strip('\n'), ref_file.readlines()))]

print(len(ref[0]))
ref[0][:10]

1295


['and i will not repeat speaking',
 'the land was in distress',
 'he who enters the foreign lands',
 'and he who should mitigate is causing suffering',
 'whom can i talk to today',
 'it was learned of by hearsay',
 'everyone',
 'and she recounted this matter to him',
 'how good she is',
 'the season of growing']

In [27]:
pred_path = "EgyptianTranslation/onmt-files/run/pred_3000.txt"
with open(pred_path, 'r') as pred_file:
  pred = list(map(lambda x: x.strip('\n'), pred_file.readlines()))

print(len(pred))
pred[:10]

1295


['i will not speak again',
 'and the land is in the barn',
 'he who enters the foreign lands',
 'and he who should mitigate is causing suffering',
 'whom can i talk to today',
 'it was learned of by hearsay',
 'everyone',
 'then it means it was recounted',
 'how good she is',
 'the season of growing']

In [28]:
!pip install sacrebleu

In [74]:
from nltk.translate import bleu_score

split_ref = [item.split() for item in ref[0]]
split_pred = [item.split() for item in pred]
bleu_score.corpus_bleu(split_ref, split_pred)

/usr/local/lib/python3.7/dist-packages/nltk/translate/bleu_score.py:490: UserWarning: 
Corpus/Sentence contains 0 counts of 2-gram overlaps.
BLEU scores might be undesirable; use SmoothingFunction().
  warnings.warn(_msg)


0.4128948227098469